In [11]:
import json
from os import listdir
from tqdm import tqdm
import re
path = './sina_news_gbk' # 压缩包中未上传

In [12]:
filelist = listdir(path)
filelist.remove('README.txt')
filelist

['2016-02.txt',
 '2016-06.txt',
 '2016-08.txt',
 '2016-09.txt',
 '2016-07.txt',
 '2016-04.txt',
 '2016-05.txt',
 '2016-11.txt',
 '2016-10.txt']

In [13]:
def sentence_process(sentence, dot)->str:
    '''
    字符处理函数：
    1.将阿拉伯数字替换成相应的汉字。
    2.将所有非一二级汉字的字符替换成空格（形成“词组”）。
    函数返回处理后的字符串
    '''
    hanzi = ['零', '一', '二', '三', '四', '五', '六', '七', '八', '九']
    new_sen = ""
    num_str = "0123456789"
    for unit in sentence:
        if (num_str.find(unit)>-1):
            new_sen = new_sen + hanzi[re.search(unit, num_str).span()[1] - 1]
        elif unit in dot:
            new_sen = new_sen + " "
        else:
            new_sen = new_sen + unit
    return new_sen

In [ ]:
# 生成sentences_raw.txt
print("Read and merge content of txt")
for file in filelist:
    print(file)
    with open(path + "/" + file, "r", encoding='gbk') as f_read:
            with open("./sentences_raw.txt", "w", encoding='utf-8') as f_in:
                for line in tqdm(f_read.readlines(), desc="Processing"):
                    data = json.loads(line)
                    f_in.write(data["title"]+'\n')
                    f_in.write(data["html"].strip("原标题："+data["title"]) +'\n')

# 统计语料库中出现的符号
dot = []
with open("./一二级汉字表.txt", "r", encoding='utf-8') as f1:
    print("Find special symbols")
    word = f1.read()
    with open("./sentences_raw.txt", "r", encoding='utf-8') as f2:
        sentence = f2.read()
        for unit in tqdm(sentence, desc="Processing"):
            if (unit not in word):
                dot.append(unit)
dot = list(set(dot))

# 生成sentences.txt
with open("./sentences_raw.txt", "r", encoding='utf-8') as f_read:
    with open("./sentences.txt", "w", encoding='utf-8') as f_in:
        print("Filter special symbols")
        for line in tqdm(f_read.readlines(), desc="Processing"):
            f_in.write(sentence_process(line, dot)+'\n')

# 规整数据格式，“一句话”为一行
with open("./sentences.txt", "r", encoding='utf-8') as f_read:
    with open("./sentences_normal.txt", "w", encoding='utf-8') as f_in:
        print("Adjust the data format")
        lines = f_read.readlines() # 整行读取数据，函数返回一个字符串
        for line in tqdm(lines, desc="Processing"):
            if line:
                items = line.split()
                for item in items:
                    f_in.write(item + "\n")